# Mobile click fraud detection using Spark ML and Scikit-Learn

### Brian Kim - March 2018




In [60]:
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests, os, sy,math
import pyspark
import itertools
import re
import numpy as np
import pickle 
import patsy
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pylab as pl
from sklearn.metrics import precision_score
from sklearn import preprocessing
from sklearn import feature_selection as f_select
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
% matplotlib inline
warnings.filterwarnings('ignore')
sns.set_style("darkgrid")

from sklearn.metrics import roc_curve, auc
from __future__ import print_function, division



from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV



Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/usr/spark-2.1.1/python/pyspark/ml/wrapper.py", line 76, in __del__
    SparkContext._active_spark_context._gateway.detach(self._java_obj)
AttributeError: 'RandomForestClassifier' object has no attribute '_java_obj'


In [2]:
pyspark.__version__

'2.1.1+without.hadoop'

## Connect to the Spark server

We will be using the SparkSession (`spark`) to access our spark cluster..

In [3]:
# spark = pyspark.sql.SparkSession.builder. shift tab 
spark = pyspark.sql.SparkSession.builder.getOrCreate()

# SparkSession (supplanted spark context) is a conduit to access all spark functionality, 
# ONLY one single sparksession exists per JVM

In [4]:
dfx= pd.read_pickle('mobile/click_data/dfx.pickle')

sd= spark.read.csv('mobile/click_data/dfx.csv',sep=',',
                     inferSchema=True,
                     header=True)

In [5]:
X= dfx[['avg_per_min', 'max_per_min', 'var_per_min',
       'skew_per_min', 'avg_per_fivemin', 'max_per_fivemin', 'var_per_fivemin',
       'skew_per_fivemin', 'avg_per_hour', 'max_per_hour', 'var_per_hour',
       'skew_per_hour', 'avg_per_3h', 'max_per_3h', 'var_per_3h',
       'skew_per_3h', 'avg_sixhour', 'max_sixhour', 'var_sixhour',
       'skew_sixhour', 'click_count', 'max_same_ip_ct', 'ipvar', 'unq_ip_ct',
       'ip_click_ratio', 'max_same_agent_ct', 'agent_var', 'unq_agent_ct',
       'agent_click_ratio', 'max_same_cid_ct', 'cid_var', 'unq_cid_ct',
       'cid_click_ratio', 'max_same_cntr_ct', 'cntr_var', 'unq_cntr_ct',
       'cntr_click_ratio', 'unq_referer_ct', 'referer_click_ratio']]
y = dfx[['status']]

In [6]:
y.status.value_counts()

0    6369
1     181
Name: status, dtype: int64

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sm = SMOTE(random_state=42, ratio = 1.0)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)
Xrs=X_train_res
yrs=y_train_res
# Xrs = pd.DataFrame(Xrs, columns=['Xrs']).to_csv('mobile/click_data/Xrs.csv')
Xtestrs= X_test
ytestrs= y_test


# yrs = pd.DataFrame(Xrs, columns=['yrs']).to_csv('mobile/click_data/yrs.csv')

In [8]:
Xrs

array([[1.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        3.33333333e-01, 3.00000000e+00, 5.00000000e-01],
       [1.26643599e+00, 5.00000000e+00, 3.11676870e-01, ...,
        1.33576199e-02, 4.28000000e+02, 1.29933212e-01],
       [1.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 5.00000000e-01],
       ...,
       [1.75573268e+00, 6.87740241e+00, 1.49032215e+00, ...,
        2.94326533e-02, 0.00000000e+00, 0.00000000e+00],
       [5.42023869e+00, 1.55627462e+01, 1.89883460e+01, ...,
        1.29600808e-02, 4.27668134e+02, 4.83333641e-01],
       [1.69076843e+00, 8.98658085e+00, 1.05437149e+00, ...,
        1.20160504e-02, 1.00812789e+03, 3.10819751e-01]])

In [9]:
train=pd.DataFrame(Xrs,columns=X.columns)
train['status']=yrs
train.to_csv('train_spark.csv')
train.head()


# train=pd.DataFrame(Xrs,columns=X.columns)

# ydf=pd.DataFrame(yrs, columns=y.columns)
# train['status']=ydf
# train.to_csv('train_spark.csv')
# train.head()

,avg_per_min,max_per_min,var_per_min,skew_per_min,avg_per_fivemin,max_per_fivemin,var_per_fivemin,skew_per_fivemin,avg_per_hour,max_per_hour,...,cid_var,unq_cid_ct,cid_click_ratio,max_same_cntr_ct,cntr_var,unq_cntr_ct,cntr_click_ratio,unq_referer_ct,referer_click_ratio,status
0,1.000000,1.0,0.000000,0.000000,1.200000,2.0,0.200000,2.236068,1.200000,2.0,...,0.000000,6.0,1.000000,4.0,2.000000,2.0,0.333333,3.0,0.500000,0
1,1.266436,5.0,0.311677,2.353842,2.393895,16.0,2.430915,1.731812,22.875000,50.0,...,2132.070590,178.0,0.054038,1984.0,94758.446089,44.0,0.013358,428.0,0.129933,0
2,1.000000,1.0,0.000000,0.000000,1.000000,1.0,0.000000,0.000000,1.000000,1.0,...,0.000000,2.0,1.000000,1.0,0.000000,2.0,1.000000,1.0,0.500000,0
3,1.059880,3.0,0.068682,4.713853,1.204082,4.0,0.245737,2.780833,2.765625,15.0,...,8.872926,52.0,0.293785,177.0,0.000000,1.0,0.005650,4.0,0.022599,0
4,1.000000,1.0,0.000000,0.000000,1.000000,1.0,0.000000,0.000000,1.500000,4.0,...,5.363636,11.0,0.458333,19.0,91.000000,3.0,0.125000,18.0,0.750000,0


In [10]:
# 50- 50 resample !
train.status.value_counts()

1    5107
0    5107
Name: status, dtype: int64

In [11]:
test=pd.DataFrame(Xtestrs,columns=X.columns)
test['status']=ytestrs
test.to_csv('test_spark.csv')
test.head()

,avg_per_min,max_per_min,var_per_min,skew_per_min,avg_per_fivemin,max_per_fivemin,var_per_fivemin,skew_per_fivemin,avg_per_hour,max_per_hour,...,cid_var,unq_cid_ct,cid_click_ratio,max_same_cntr_ct,cntr_var,unq_cntr_ct,cntr_click_ratio,unq_referer_ct,referer_click_ratio,status
1212,1.000000,1,0.000000,0.000000,1.055556,2,0.053968,4.051370,1.407407,4,...,0.132576,33,0.868421,27.0,104.666667,6,0.157895,17,0.447368,0
773,1.000000,1,0.000000,0.000000,1.142857,2,0.142857,2.645751,1.142857,2,...,0.000000,1,0.125000,3.0,1.000000,3,0.375000,1,0.125000,0
3985,1.000000,1,0.000000,0.000000,1.000000,1,0.000000,0.000000,1.000000,1,...,0.000000,1,1.000000,1.0,0.000000,1,1.000000,1,1.000000,0
782,1.000000,1,0.000000,0.000000,1.000000,1,0.000000,0.000000,1.000000,1,...,0.000000,1,1.000000,1.0,0.000000,1,1.000000,1,1.000000,0
6044,1.041958,2,0.040481,4.617729,1.252101,4,0.325737,2.736543,2.921569,11,...,17.537821,40,0.268456,36.0,110.079051,23,0.154362,73,0.489933,0


In [12]:
# read in train into spark

train=spark.read.csv('train_spark.csv', sep= ',',inferSchema=None, header=True)
test = spark.read.csv('test_spark.csv',sep= ',',inferSchema=None, header=True)

In [13]:
from pyspark.sql.types import StringType, DoubleType, StructType, StructField
type(sd)

pyspark.sql.dataframe.DataFrame

In [14]:
def preview(df, n=3):
    return pd.DataFrame(df.take(n), columns=df.columns)

preview(train)

,_c0,avg_per_min,max_per_min,var_per_min,skew_per_min,avg_per_fivemin,max_per_fivemin,var_per_fivemin,skew_per_fivemin,avg_per_hour,...,cid_var,unq_cid_ct,cid_click_ratio,max_same_cntr_ct,cntr_var,unq_cntr_ct,cntr_click_ratio,unq_referer_ct,referer_click_ratio,status
0,0,1.0,1.0,0.0,0.0,1.2,2.0,0.2,2.23606797749979,1.2,...,0.0,6.0,1.0,4.0,2.0,2.0,0.3333333333333333,3.0,0.5,0
1,1,1.2664359861591696,5.0,0.3116768698429595,2.3538424130738558,2.3938953488372094,16.0,2.4309154334038108,1.7318120303736304,22.875,...,2132.070589728942,178.0,0.054037644201578625,1984.0,94758.44608879491,44.0,0.013357619914996965,428.0,0.129933211900425,0
2,2,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,2.0,1.0,1.0,0.0,2.0,1.0,1.0,0.5,0


In [15]:
test=test.drop('_c0', 'partnerid')

train= train.drop('_c0', 'partnerid')

In [16]:
test.printSchema()

root
 |-- avg_per_min: string (nullable = true)
 |-- max_per_min: string (nullable = true)
 |-- var_per_min: string (nullable = true)
 |-- skew_per_min: string (nullable = true)
 |-- avg_per_fivemin: string (nullable = true)
 |-- max_per_fivemin: string (nullable = true)
 |-- var_per_fivemin: string (nullable = true)
 |-- skew_per_fivemin: string (nullable = true)
 |-- avg_per_hour: string (nullable = true)
 |-- max_per_hour: string (nullable = true)
 |-- var_per_hour: string (nullable = true)
 |-- skew_per_hour: string (nullable = true)
 |-- avg_per_3h: string (nullable = true)
 |-- max_per_3h: string (nullable = true)
 |-- var_per_3h: string (nullable = true)
 |-- skew_per_3h: string (nullable = true)
 |-- avg_sixhour: string (nullable = true)
 |-- max_sixhour: string (nullable = true)
 |-- var_sixhour: string (nullable = true)
 |-- skew_sixhour: string (nullable = true)
 |-- click_count: string (nullable = true)
 |-- max_same_ip_ct: string (nullable = true)
 |-- ipvar: string (nulla

In [17]:
sd.registerTempTable('sd')

In [18]:
ok_ct = spark.sql(r"""select status, count(status) as total from
                sd group by status""")
ok_ct.show()

+------+-----+
|status|total|
+------+-----+
|     1|  181|
|     0| 6369|
+------+-----+



In [19]:
def preview(df, n=10):
    return pd.DataFrame(df.take(n), columns=df.columns)

preview(train)

,avg_per_min,max_per_min,var_per_min,skew_per_min,avg_per_fivemin,max_per_fivemin,var_per_fivemin,skew_per_fivemin,avg_per_hour,max_per_hour,...,cid_var,unq_cid_ct,cid_click_ratio,max_same_cntr_ct,cntr_var,unq_cntr_ct,cntr_click_ratio,unq_referer_ct,referer_click_ratio,status
0,1.0,1.0,0.0,0.0,1.2,2.0,0.2,2.23606797749979,1.2,2.0,...,0.0,6.0,1.0,4.0,2.0,2.0,0.3333333333333333,3.0,0.5,0
1,1.2664359861591696,5.0,0.3116768698429595,2.3538424130738558,2.3938953488372094,16.0,2.4309154334038108,1.7318120303736304,22.875,50.0,...,2132.070589728942,178.0,0.054037644201578625,1984.0,94758.44608879491,44.0,0.013357619914996965,428.0,0.129933211900425,0
2,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,2.0,1.0,1.0,0.0,2.0,1.0,1.0,0.5,0
3,1.0598802395209581,3.0,0.06868191328186994,4.7138530333051545,1.2040816326530612,4.0,0.2457366508247135,2.780833413357422,2.765625,15.0,...,8.87292609351433,52.0,0.2937853107344633,177.0,0.0,1.0,0.005649717514124294,4.0,0.022598870056497175,0
4,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.5,4.0,...,5.363636363636364,11.0,0.4583333333333333,19.0,91.0,3.0,0.125,18.0,0.75,0
5,1.6808510638297873,7.0,1.9185541066117588,2.405669029226983,2.0256410256410255,7.0,3.0123210123210113,1.6881033794666018,2.925925925925926,21.0,...,1.3452896310039169,99.0,0.6265822784810127,56.0,235.59558823529412,17.0,0.10759493670886076,30.0,0.189873417721519,0
6,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0
7,1.1892744479495267,3.0,0.1665934592500898,1.8714642512011272,2.244047619047619,7.0,1.7185272312517816,1.2331904805694898,10.18918918918919,36.0,...,69.09313725490195,52.0,0.13793103448275862,112.0,2027.861111111111,9.0,0.023872679045092837,35.0,0.09283819628647215,1
8,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0
9,1.3333333333333333,3.0,0.6666666666666667,2.4494897427831783,1.3333333333333333,3.0,0.6666666666666667,2.4494897427831783,1.3333333333333333,3.0,...,2.0,4.0,0.5,8.0,0.0,1.0,0.125,5.0,0.625,0


In [20]:
from pyspark.sql.functions import udf

In [21]:
def trim(string):
    return (string
            .strip()# string columns have spaces due to csv file format
            )       
            
    
    
trim = udf(trim)

for dtype in train.dtypes: # dtypes : returns tuples,: ('account_length', 'int')
    column = dtype[0]
    if dtype[1] == 'string':
        train = train.withColumn(column, trim(train[column]))

        
preview(train)

,avg_per_min,max_per_min,var_per_min,skew_per_min,avg_per_fivemin,max_per_fivemin,var_per_fivemin,skew_per_fivemin,avg_per_hour,max_per_hour,...,cid_var,unq_cid_ct,cid_click_ratio,max_same_cntr_ct,cntr_var,unq_cntr_ct,cntr_click_ratio,unq_referer_ct,referer_click_ratio,status
0,1.0,1.0,0.0,0.0,1.2,2.0,0.2,2.23606797749979,1.2,2.0,...,0.0,6.0,1.0,4.0,2.0,2.0,0.3333333333333333,3.0,0.5,0
1,1.2664359861591696,5.0,0.3116768698429595,2.3538424130738558,2.3938953488372094,16.0,2.4309154334038108,1.7318120303736304,22.875,50.0,...,2132.070589728942,178.0,0.054037644201578625,1984.0,94758.44608879491,44.0,0.013357619914996965,428.0,0.129933211900425,0
2,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,2.0,1.0,1.0,0.0,2.0,1.0,1.0,0.5,0
3,1.0598802395209581,3.0,0.06868191328186994,4.7138530333051545,1.2040816326530612,4.0,0.2457366508247135,2.780833413357422,2.765625,15.0,...,8.87292609351433,52.0,0.2937853107344633,177.0,0.0,1.0,0.005649717514124294,4.0,0.022598870056497175,0
4,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.5,4.0,...,5.363636363636364,11.0,0.4583333333333333,19.0,91.0,3.0,0.125,18.0,0.75,0
5,1.6808510638297873,7.0,1.9185541066117588,2.405669029226983,2.0256410256410255,7.0,3.0123210123210113,1.6881033794666018,2.925925925925926,21.0,...,1.3452896310039169,99.0,0.6265822784810127,56.0,235.59558823529412,17.0,0.10759493670886076,30.0,0.189873417721519,0
6,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0
7,1.1892744479495267,3.0,0.1665934592500898,1.8714642512011272,2.244047619047619,7.0,1.7185272312517816,1.2331904805694898,10.18918918918919,36.0,...,69.09313725490195,52.0,0.13793103448275862,112.0,2027.861111111111,9.0,0.023872679045092837,35.0,0.09283819628647215,1
8,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0
9,1.3333333333333333,3.0,0.6666666666666667,2.4494897427831783,1.3333333333333333,3.0,0.6666666666666667,2.4494897427831783,1.3333333333333333,3.0,...,2.0,4.0,0.5,8.0,0.0,1.0,0.125,5.0,0.625,0


In [22]:
for dtype in test.dtypes: # dtypes : returns tuples,: ('account_length', 'int')
    column = dtype[0]
    if dtype[1] == 'string':
        test = test.withColumn(column, trim(test[column]))




In [23]:
#predictors have to be of type double for many of Spark's ML models
# boolean types converted to integers
for column in train.columns:
    train = train.withColumn(column, train[column].cast('double'))
    

train = train.withColumn('status', train['status'].cast('boolean').cast('double'))




for column in test.columns:
    test = test.withColumn(column, test[column].cast('double'))
    
test = test.withColumn('status', test['status'].cast('double'))

In [24]:
preview(test)

,avg_per_min,max_per_min,var_per_min,skew_per_min,avg_per_fivemin,max_per_fivemin,var_per_fivemin,skew_per_fivemin,avg_per_hour,max_per_hour,...,cid_var,unq_cid_ct,cid_click_ratio,max_same_cntr_ct,cntr_var,unq_cntr_ct,cntr_click_ratio,unq_referer_ct,referer_click_ratio,status
0,1.000000,1.0,0.000000,0.000000,1.055556,2.0,0.053968,4.051370,1.407407,4.0,...,0.132576,33.0,0.868421,27.0,104.666667,6.0,0.157895,17.0,0.447368,0.0
1,1.000000,1.0,0.000000,0.000000,1.142857,2.0,0.142857,2.645751,1.142857,2.0,...,0.000000,1.0,0.125000,3.0,1.000000,3.0,0.375000,1.0,0.125000,0.0
2,1.000000,1.0,0.000000,0.000000,1.000000,1.0,0.000000,0.000000,1.000000,1.0,...,0.000000,1.0,1.000000,1.0,0.000000,1.0,1.000000,1.0,1.000000,0.0
3,1.000000,1.0,0.000000,0.000000,1.000000,1.0,0.000000,0.000000,1.000000,1.0,...,0.000000,1.0,1.000000,1.0,0.000000,1.0,1.000000,1.0,1.000000,0.0
4,1.041958,2.0,0.040481,4.617729,1.252101,4.0,0.325737,2.736543,2.921569,11.0,...,17.537821,40.0,0.268456,36.0,110.079051,23.0,0.154362,73.0,0.489933,0.0
5,1.211382,6.0,0.594296,5.071570,1.379630,11.0,1.471357,5.632933,2.811321,13.0,...,13.414305,39.0,0.261745,144.0,5064.916667,4.0,0.026846,36.0,0.241611,0.0
6,1.333600,7.0,0.557157,2.911965,2.338008,18.0,5.651039,2.868924,11.907143,90.0,...,422.477133,287.0,0.172166,960.0,15716.037006,60.0,0.035993,927.0,0.556089,0.0
7,1.000000,1.0,0.000000,0.000000,1.000000,1.0,0.000000,0.000000,1.000000,1.0,...,0.000000,1.0,0.500000,2.0,0.000000,1.0,0.500000,2.0,1.000000,0.0
8,1.027778,2.0,0.027778,6.000000,1.088235,2.0,0.082888,3.039226,1.275862,3.0,...,12.765789,20.0,0.540541,8.0,5.648352,14.0,0.378378,11.0,0.297297,0.0
9,1.000000,1.0,0.000000,0.000000,1.000000,1.0,0.000000,0.000000,1.000000,1.0,...,0.000000,2.0,1.000000,1.0,0.000000,2.0,1.000000,0.0,0.000000,0.0


In [25]:
#Train and test data must be in the same dataframe for Spark ML libraries.

from pyspark.ml.feature import VectorAssembler

# the feature columns
features = train.schema.names[:-1]
assembler = VectorAssembler(inputCols=features, outputCol='features')

train_pack = assembler.transform(train)
test_pack = assembler.transform(test)



In [26]:
def preview(df, n=10):
    return pd.DataFrame(df.take(n), columns=df.columns)



In [27]:
preview(train_pack)

,avg_per_min,max_per_min,var_per_min,skew_per_min,avg_per_fivemin,max_per_fivemin,var_per_fivemin,skew_per_fivemin,avg_per_hour,max_per_hour,...,unq_cid_ct,cid_click_ratio,max_same_cntr_ct,cntr_var,unq_cntr_ct,cntr_click_ratio,unq_referer_ct,referer_click_ratio,status,features
0,1.000000,1.0,0.000000,0.000000,1.200000,2.0,0.200000,2.236068,1.200000,2.0,...,6.0,1.000000,4.0,2.000000,2.0,0.333333,3.0,0.500000,0.0,"[1.0, 1.0, 0.0, 0.0, 1.2, 2.0, 0.2, 2.23606797..."
1,1.266436,5.0,0.311677,2.353842,2.393895,16.0,2.430915,1.731812,22.875000,50.0,...,178.0,0.054038,1984.0,94758.446089,44.0,0.013358,428.0,0.129933,0.0,"[1.2664359861591696, 5.0, 0.3116768698429595, ..."
2,1.000000,1.0,0.000000,0.000000,1.000000,1.0,0.000000,0.000000,1.000000,1.0,...,2.0,1.000000,1.0,0.000000,2.0,1.000000,1.0,0.500000,0.0,"[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ..."
3,1.059880,3.0,0.068682,4.713853,1.204082,4.0,0.245737,2.780833,2.765625,15.0,...,52.0,0.293785,177.0,0.000000,1.0,0.005650,4.0,0.022599,0.0,"[1.0598802395209581, 3.0, 0.06868191328186994,..."
4,1.000000,1.0,0.000000,0.000000,1.000000,1.0,0.000000,0.000000,1.500000,4.0,...,11.0,0.458333,19.0,91.000000,3.0,0.125000,18.0,0.750000,0.0,"[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.5, ..."
5,1.680851,7.0,1.918554,2.405669,2.025641,7.0,3.012321,1.688103,2.925926,21.0,...,99.0,0.626582,56.0,235.595588,17.0,0.107595,30.0,0.189873,0.0,"[1.6808510638297873, 7.0, 1.9185541066117588, ..."
6,1.000000,1.0,0.000000,0.000000,1.000000,1.0,0.000000,0.000000,1.000000,1.0,...,1.0,1.000000,1.0,0.000000,1.0,1.000000,0.0,0.000000,0.0,"(1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ..."
7,1.189274,3.0,0.166593,1.871464,2.244048,7.0,1.718527,1.233190,10.189189,36.0,...,52.0,0.137931,112.0,2027.861111,9.0,0.023873,35.0,0.092838,1.0,"[1.1892744479495267, 3.0, 0.1665934592500898, ..."
8,1.000000,1.0,0.000000,0.000000,1.000000,1.0,0.000000,0.000000,1.000000,1.0,...,1.0,1.000000,1.0,0.000000,1.0,1.000000,0.0,0.000000,0.0,"(1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ..."
9,1.333333,3.0,0.666667,2.449490,1.333333,3.0,0.666667,2.449490,1.333333,3.0,...,4.0,0.500000,8.0,0.000000,1.0,0.125000,5.0,0.625000,0.0,"[1.3333333333333333, 3.0, 0.6666666666666667, ..."


## Data is now ready to be analyzed with Spark's ML libraries

In [31]:
from pyspark.ml.feature import MinMaxScaler

minmaxscale = MinMaxScaler(inputCol='features', outputCol='features_scaled')
minmaxscale = minmaxscale.fit(train_pack)

train_pack = minmaxscale.transform(train_pack)
test_pack  = minmaxscale.transform(test_pack)

IllegalArgumentException: 'requirement failed: Output column features_scaled already exists.'

In [32]:
preview(test_pack)

,avg_per_min,max_per_min,var_per_min,skew_per_min,avg_per_fivemin,max_per_fivemin,var_per_fivemin,skew_per_fivemin,avg_per_hour,max_per_hour,...,cid_click_ratio,max_same_cntr_ct,cntr_var,unq_cntr_ct,cntr_click_ratio,unq_referer_ct,referer_click_ratio,status,features,features_scaled
0,1.000000,1.0,0.000000,0.000000,1.055556,2.0,0.053968,4.051370,1.407407,4.0,...,0.868421,27.0,104.666667,6.0,0.157895,17.0,0.447368,0.0,"[1.0, 1.0, 0.0, 0.0, 1.0555555555555556, 2.0, ...","[0.0, 0.0, 0.0, 0.04785437581153394, 0.0001692..."
1,1.000000,1.0,0.000000,0.000000,1.142857,2.0,0.142857,2.645751,1.142857,2.0,...,0.125000,3.0,1.000000,3.0,0.375000,1.0,0.125000,0.0,"[1.0, 1.0, 0.0, 0.0, 1.1428571428571428, 2.0, ...","[0.0, 0.0, 0.0, 0.04785437581153394, 0.0004351..."
2,1.000000,1.0,0.000000,0.000000,1.000000,1.0,0.000000,0.000000,1.000000,1.0,...,1.000000,1.0,0.000000,1.0,1.000000,1.0,1.000000,0.0,"[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.04785437581153394, 0.0, 0.0,..."
3,1.000000,1.0,0.000000,0.000000,1.000000,1.0,0.000000,0.000000,1.000000,1.0,...,1.000000,1.0,0.000000,1.0,1.000000,1.0,1.000000,0.0,"[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.04785437581153394, 0.0, 0.0,..."
4,1.041958,2.0,0.040481,4.617729,1.252101,4.0,0.325737,2.736543,2.921569,11.0,...,0.268456,36.0,110.079051,23.0,0.154362,73.0,0.489933,0.0,"[1.0419580419580419, 2.0, 0.04048064611444891,...","[0.000646840949958304, 0.002188183807439825, 1..."
5,1.211382,6.0,0.594296,5.071570,1.379630,11.0,1.471357,5.632933,2.811321,13.0,...,0.261745,144.0,5064.916667,4.0,0.026846,36.0,0.241611,0.0,"[1.2113821138211383, 6.0, 0.5942956150872979, ...","[0.0032587461408468557, 0.010940919037199124, ..."
6,1.333600,7.0,0.557157,2.911965,2.338008,18.0,5.651039,2.868924,11.907143,90.0,...,0.172166,960.0,15716.037006,60.0,0.035993,927.0,0.556089,0.0,"[1.3336, 7.0, 0.5571567654123302, 2.9119646517...","[0.005142903024928492, 0.01312910284463895, 0...."
7,1.000000,1.0,0.000000,0.000000,1.000000,1.0,0.000000,0.000000,1.000000,1.0,...,0.500000,2.0,0.000000,1.0,0.500000,2.0,1.000000,0.0,"[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.04785437581153394, 0.0, 0.0,..."
8,1.027778,2.0,0.027778,6.000000,1.088235,2.0,0.082888,3.039226,1.275862,3.0,...,0.540541,8.0,5.648352,14.0,0.378378,11.0,0.297297,0.0,"[1.0277777777777777, 2.0, 0.027777777777777773...","[0.0004282326659446173, 0.002188183807439825, ..."
9,1.000000,1.0,0.000000,0.000000,1.000000,1.0,0.000000,0.000000,1.000000,1.0,...,1.000000,1.0,0.000000,2.0,1.000000,0.0,0.000000,0.0,"(1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.04785437581153394, 0.0, 0.0,..."


In [33]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, \
                                      GBTClassifier, LogisticRegression

In [34]:
# logistic regression
lr = LogisticRegression(labelCol='status', 
                        featuresCol='features_scaled',  # train scaled features
                        predictionCol='prediction')

lr_model = lr.fit(train_pack)
lr_pred = lr_model.transform(test_pack)  # returns another dataframe, see below!

# Decision Tree (depth = 2)
dt = DecisionTreeClassifier(maxDepth=2, 
                            labelCol='status', 
                            featuresCol='features',   
                            predictionCol='prediction')

dt_model = dt.fit(train_pack)
dt_pred = dt_model.transform(test_pack)

# Random Forest (also of depth = 2)
rf = RandomForestClassifier(maxDepth=2, 
                            labelCol='status',
                            featuresCol='features',
                            predictionCol='prediction')

rf_model = rf.fit(train_pack)
rf_pred = rf_model.transform(test_pack)

# Gradient Boosted Tree (depth = 2)
gbt = GBTClassifier(maxDepth=2, 
                    labelCol='status', 
                    featuresCol='features',
                    predictionCol='prediction')

gbt_model = gbt.fit(train_pack)
gbt_pred = gbt_model.transform(test_pack)

In [35]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, \
                                  MulticlassClassificationEvaluator

accuracy = MulticlassClassificationEvaluator(labelCol='status', 
                                             predictionCol='prediction',
                                             metricName='accuracy')

precision = MulticlassClassificationEvaluator(labelCol='status', 
                                              predictionCol='prediction',
                                              metricName='weightedPrecision')

recall = MulticlassClassificationEvaluator(labelCol='status', 
                                           predictionCol='prediction',
                                           metricName='weightedRecall')

f1 = MulticlassClassificationEvaluator(labelCol='status', 
                                       predictionCol='prediction',
                                       metricName='f1')

areaROC = BinaryClassificationEvaluator(labelCol='status',
                                        rawPredictionCol='prediction',
                                        metricName='areaUnderROC')

areaPR = BinaryClassificationEvaluator(labelCol='status',
                                       rawPredictionCol='prediction',
                                       metricName='areaUnderPR')

In [36]:
# the error metrics
metrics = [accuracy, precision, recall, f1, areaROC, areaPR]
metric_labels = ['accuracy', 'precision', 'recall', 'f1', 'areaROC', 'areaPR']

# the predictions from each model
predictions = [lr_pred, dt_pred, rf_pred, gbt_pred]
predict_labels = ['LR', 'DT', 'RF', 'GBT']

eval_list = list()

# for each model's predictions, calculate error metrics
# and add to a Pandas series
for pred in zip(predict_labels, predictions):
    name = pred[0]
    predict = pred[1]
    
    metric_vals = pd.Series(dict([(x[0], x[1].evaluate(predict))   # ex: accuracy.evaluate(lr_pred) 
                                 for x in zip(metric_labels, metrics)]),
                            name=name)
    eval_list.append(metric_vals)
    
    
eval_list[0]

accuracy     0.835878
areaPR       0.557444
areaROC      0.884756
f1           0.884701
precision    0.967030
recall       0.835878
Name: LR, dtype: float64

In [37]:
# combine all the series into a dataframe
eval_df = pd.concat(eval_list, axis=1).T
eval_df

,accuracy,areaPR,areaROC,f1,precision,recall
LR,0.835878,0.557444,0.884756,0.884701,0.967030,0.835878
DT,0.804580,0.567780,0.888553,0.864556,0.968106,0.804580
RF,0.821374,0.562664,0.887249,0.875432,0.967555,0.821374
GBT,0.881679,0.561224,0.888487,0.913668,0.967210,0.881679


In [38]:
dfx.columns

Index(['partnerid', 'avg_per_min', 'max_per_min', 'var_per_min',
       'skew_per_min', 'avg_per_fivemin', 'max_per_fivemin', 'var_per_fivemin',
       'skew_per_fivemin', 'avg_per_hour', 'max_per_hour', 'var_per_hour',
       'skew_per_hour', 'avg_per_3h', 'max_per_3h', 'var_per_3h',
       'skew_per_3h', 'avg_sixhour', 'max_sixhour', 'var_sixhour',
       'skew_sixhour', 'click_count', 'max_same_ip_ct', 'ipvar', 'unq_ip_ct',
       'ip_click_ratio', 'max_same_agent_ct', 'agent_var', 'unq_agent_ct',
       'agent_click_ratio', 'max_same_cid_ct', 'cid_var', 'unq_cid_ct',
       'cid_click_ratio', 'max_same_cntr_ct', 'cntr_var', 'unq_cntr_ct',
       'cntr_click_ratio', 'unq_referer_ct', 'referer_click_ratio', 'status'],
      dtype='object')

## Scikit Learn

In [61]:
dfx= pd.read_pickle('mobile/click_data/dfx.pickle')
X= dfx[['avg_per_min', 'max_per_min', 'var_per_min',
       'skew_per_min', 'avg_per_fivemin', 'max_per_fivemin', 'var_per_fivemin',
       'skew_per_fivemin', 'avg_per_hour', 'max_per_hour', 'var_per_hour',
       'skew_per_hour', 'avg_per_3h', 'max_per_3h', 'var_per_3h',
       'skew_per_3h', 'avg_sixhour', 'max_sixhour', 'var_sixhour',
       'skew_sixhour', 'click_count', 'max_same_ip_ct', 'ipvar', 'unq_ip_ct',
       'ip_click_ratio', 'max_same_agent_ct', 'agent_var', 'unq_agent_ct',
       'agent_click_ratio', 'max_same_cid_ct', 'cid_var', 'unq_cid_ct',
       'cid_click_ratio', 'max_same_cntr_ct', 'cntr_var', 'unq_cntr_ct',
       'cntr_click_ratio', 'unq_referer_ct', 'referer_click_ratio']]
y = dfx.status

In [62]:
y.dtypes

dtype('int64')

In [63]:
##Resample using SMOTE to address imbalanced class

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sm = SMOTE(random_state=42, ratio = 1.0)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)
Xr=X_train_res
yr=y_train_res

In [64]:
Xr

array([[1.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        3.33333333e-01, 3.00000000e+00, 5.00000000e-01],
       [1.26643599e+00, 5.00000000e+00, 3.11676870e-01, ...,
        1.33576199e-02, 4.28000000e+02, 1.29933212e-01],
       [1.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 5.00000000e-01],
       ...,
       [1.75573268e+00, 6.87740241e+00, 1.49032215e+00, ...,
        2.94326533e-02, 0.00000000e+00, 0.00000000e+00],
       [5.42023869e+00, 1.55627462e+01, 1.89883460e+01, ...,
        1.29600808e-02, 4.27668134e+02, 4.83333641e-01],
       [1.69076843e+00, 8.98658085e+00, 1.05437149e+00, ...,
        1.20160504e-02, 1.00812789e+03, 3.10819751e-01]])

In [65]:
rfsmote = RandomForestClassifier(n_estimators=25, random_state=12)
rfsmote.fit(X_train_res, y_train_res)

rfsmote_pred = rfsmote.predict(X_test)
accuracy = accuracy_score(y_test, rfsmote_pred)

In [68]:


from sklearn.metrics import classification_report
print('Random Forest with SMOTE')
from sklearn.metrics import accuracy_score
print(classification_report(y_test, rfsmote_pred))
print (accuracy)
print ('Validation Results for Random Forest')

print (accuracy_score(y_test, rfsmote.predict(X_test)))
print ('Recall:')
print (recall_score(y_test, rfsmote.predict(X_test)))
print ('Precision')
print (precision_score(y_test, rfsmote.predict(X_test)))

Random Forest with SMOTE
             precision    recall  f1-score   support

          0       0.99      0.98      0.98      1262
          1       0.49      0.62      0.55        48

avg / total       0.97      0.96      0.96      1310

0.9625954198473282
Validation Results for Random Forest
0.9625954198473282
Recall:
0.625
Precision
0.4918032786885246


In [33]:
fpr

array([0.00000000e+00, 7.92393027e-04, 1.58478605e-03, 1.58478605e-03,
       3.96196513e-03, 4.75435816e-03, 4.75435816e-03, 6.33914422e-03,
       7.92393027e-03, 1.10935024e-02, 1.42630745e-02, 1.82250396e-02,
       2.29793978e-02, 2.45641838e-02, 2.77337559e-02, 3.40729002e-02,
       4.19968304e-02, 4.83359746e-02, 5.62599049e-02, 6.65610143e-02,
       7.92393027e-02, 1.09350238e-01, 1.33122029e-01, 1.60063391e-01,
       2.17115689e-01, 3.04278922e-01, 1.00000000e+00])

In [69]:
rf = RandomForestClassifier(n_estimators=25, random_state=12)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
print('Random Forest without SMOTE')

print(classification_report(y_test, rf_pred))

print ('Validation Results for Random Forest')

print (accuracy_score(y_test, rf.predict(X_test)))
print ('Recall:')
print (recall_score(y_test, rf.predict(X_test)))
print ('Precision')
print (precision_score(y_test, rf.predict(X_test)))

Random Forest without SMOTE
             precision    recall  f1-score   support

          0       0.97      1.00      0.98      1262
          1       0.57      0.17      0.26        48

avg / total       0.95      0.96      0.96      1310

Validation Results for Random Forest
0.9648854961832061
Recall:
0.16666666666666666
Precision
0.5714285714285714


In [70]:
##K-FOLD AND CROSS-VALIDATION

from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
kfold = model_selection.KFold(n_splits=10, random_state=42)
lrCV = LogisticRegression(class_weight='balanced')
scoring = 'accuracy'
results = model_selection.cross_val_score(lrCV, Xr,
                                          yr, cv=kfold, scoring=scoring)
print("Logistic regression with 10-fold cross validation average accuracy: %.3f" % (results.mean()))


Logistic regression with 10-fold cross validation average accuracy: 0.590


In [71]:
import xgboost as xgb
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

gbm = xgb.XGBClassifier( 
                       n_estimators=300, #arbitrary large number
                       max_depth=3,
                       learning_rate=.1, 
                       subsample=1,
                       min_child_weight=1,
                       colsample_bytree=.8
                      )
fit_model = gbm.fit(X_train,y_train, verbose = True)

In [72]:
gbm_pred =fit_model.predict(X_test)
#y_pred = model.predict(X_test)
predictions = [round(value) for value in gbm_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 97.02%


In [73]:
print(classification_report(y_test, gbm_pred))

             precision    recall  f1-score   support

          0       0.97      1.00      0.98      1262
          1       0.71      0.31      0.43        48

avg / total       0.96      0.97      0.96      1310



In [43]:
gb_pred =fit_model.predict(X_test.as_matrix())


In [80]:
#accuracy_score(y_true, y_pred.round(), normalize=False)

print('GBM without SMOTE')
accuracy = accuracy_score(y_test, gbm_pred.round(), normalize = False)
print("Accuracy: %.2f%%" % (accuracy * .01))

GBM without SMOTE
Accuracy: 12.71%


In [81]:
features = X.columns
feature_importances = rfsmote.feature_importances_
features_df = pd.DataFrame({'Features': features, 'Importance Score': feature_importances})
features_df.sort_values('Importance Score', inplace=True, ascending=False)
features_df.head(14)

,Features,Importance Score
4,avg_per_fivemin,0.100116
16,avg_sixhour,0.093184
5,max_per_fivemin,0.076633
8,avg_per_hour,0.073116
17,max_sixhour,0.056056
9,max_per_hour,0.053999
13,max_per_3h,0.051060
6,var_per_fivemin,0.048147
18,var_sixhour,0.042367
2,var_per_min,0.032151
